# Notebook: bronze_ingestão_CSV.

Este notebook foi **desenvolvido, executado e validado na plataforma Databricks Free Edition**, como parte do **MVP (Minimum Viable Product) da Sprint Engenharia de Dados**.

---

#1. Configuração dos paths 

In [0]:
%python
path_csv = "/Volumes/mvp_datasus/bronze/raw_data/"
path_bronze = "dbfs:/mvp_datasus/bronze/tables_delta/"

#2. Lista de todos os arquivos CSV

In [0]:
%python
arquivos_csv = dbutils.fs.ls(path_csv)

csvs = [f for f in arquivos_csv if f.name.endswith(".csv")]

print(f"Total de CSVs encontrados: {len(csvs)}")
for f in csvs:
    print(f"- {f.name}")


Total de CSVs encontrados: 22
- obitos_geral.csv
- obitos_hiv_faixa_0-10.csv
- obitos_hiv_faixa_0-10_fem.csv
- obitos_hiv_faixa_0-10_masc.csv
- obitos_hiv_faixa_10-25.csv
- obitos_hiv_faixa_10-25_fem.csv
- obitos_hiv_faixa_10-25_masc.csv
- obitos_hiv_faixa_25-40.csv
- obitos_hiv_faixa_25-40_fem.csv
- obitos_hiv_faixa_25-40_masc.csv
- obitos_hiv_faixa_40-55.csv
- obitos_hiv_faixa_40-55_fem.csv
- obitos_hiv_faixa_40-55_masc.csv
- obitos_hiv_faixa_55-65_fem.csv
- obitos_hiv_faixa_55-65_masc.csv
- obitos_hiv_faixa_55_65.csv
- obitos_hiv_faixa_65_mais.csv
- obitos_hiv_faixa_65_mais_fem.csv
- obitos_hiv_faixa_65_mais_masc.csv
- obitos_hiv_fem.csv
- obitos_hiv_geral.csv
- obitos_hiv_masc.csv


#3. Função para limpar nomes de colunas

In [0]:
%python
import re

def limpar_nomes(df):
    for col in df.columns:
        novo = re.sub(r'[^a-zA-Z0-9_]', '_', col)
        df = df.withColumnRenamed(col, novo)
    return df


#4. Ler CSV → tratar → salvar Delta na Bronze

In [0]:
%python
import re

def limpar_nome_tabela(nome):
    return re.sub(r'[^0-9a-zA-Z_]', '_', nome).lower()

def make_unique_columns(cols):
    seen = {}
    result = []
    for col in cols:
        col_clean = re.sub(r'[^a-zA-Z0-9_]', '_', col)
        if col_clean in seen:
            seen[col_clean] += 1
            col_clean = f"{col_clean}_{seen[col_clean]}"
        else:
            seen[col_clean] = 0
        result.append(col_clean)
    return result

target_table_schema = "mvp_datasus.bronze"

for arq in csvs:
    print("="*70)
    nome_arquivo = arq.name
    nome_tabela = nome_arquivo.replace(".csv", "")
    nome_tabela_sanitizada = limpar_nome_tabela(nome_tabela)
    tabela_full = f"{target_table_schema}.{nome_tabela_sanitizada}"

    print(f"Processando arquivo: {nome_arquivo} -> tabela: {tabela_full}")

    df = (
        spark.read
        .csv(
            arq.path,
            header=True,
            inferSchema=True,
            sep=";"
        )
    )

 
    unique_cols = make_unique_columns(df.columns)
    for old, new in zip(df.columns, unique_cols):
        if old != new:
            df = df.withColumnRenamed(old, new)

    df.write.format("delta").mode("overwrite").saveAsTable(tabela_full)

    print(f"Tabela criada/atualizada: {tabela_full}")



Processando arquivo: obitos_geral.csv -> tabela: mvp_datasus.bronze.obitos_geral
Tabela criada/atualizada: mvp_datasus.bronze.obitos_geral
Processando arquivo: obitos_hiv_faixa_0-10.csv -> tabela: mvp_datasus.bronze.obitos_hiv_faixa_0_10
Tabela criada/atualizada: mvp_datasus.bronze.obitos_hiv_faixa_0_10
Processando arquivo: obitos_hiv_faixa_0-10_fem.csv -> tabela: mvp_datasus.bronze.obitos_hiv_faixa_0_10_fem
Tabela criada/atualizada: mvp_datasus.bronze.obitos_hiv_faixa_0_10_fem
Processando arquivo: obitos_hiv_faixa_0-10_masc.csv -> tabela: mvp_datasus.bronze.obitos_hiv_faixa_0_10_masc
Tabela criada/atualizada: mvp_datasus.bronze.obitos_hiv_faixa_0_10_masc
Processando arquivo: obitos_hiv_faixa_10-25.csv -> tabela: mvp_datasus.bronze.obitos_hiv_faixa_10_25
Tabela criada/atualizada: mvp_datasus.bronze.obitos_hiv_faixa_10_25
Processando arquivo: obitos_hiv_faixa_10-25_fem.csv -> tabela: mvp_datasus.bronze.obitos_hiv_faixa_10_25_fem
Tabela criada/atualizada: mvp_datasus.bronze.obitos_hiv_fa

#5. Lista das tabelas Delta criadas (20 primeiras)

In [0]:
%python
spark.sql("SHOW TABLES IN mvp_datasus.bronze").show(truncate=False)


+--------+-----------------------------+-----------+
|database|tableName                    |isTemporary|
+--------+-----------------------------+-----------+
|bronze  |obitos_geral                 |false      |
|bronze  |obitos_hiv_faixa_0_10        |false      |
|bronze  |obitos_hiv_faixa_0_10_fem    |false      |
|bronze  |obitos_hiv_faixa_0_10_masc   |false      |
|bronze  |obitos_hiv_faixa_10_25       |false      |
|bronze  |obitos_hiv_faixa_10_25_fem   |false      |
|bronze  |obitos_hiv_faixa_10_25_masc  |false      |
|bronze  |obitos_hiv_faixa_25_40       |false      |
|bronze  |obitos_hiv_faixa_25_40_fem   |false      |
|bronze  |obitos_hiv_faixa_25_40_masc  |false      |
|bronze  |obitos_hiv_faixa_40_55       |false      |
|bronze  |obitos_hiv_faixa_40_55_fem   |false      |
|bronze  |obitos_hiv_faixa_40_55_masc  |false      |
|bronze  |obitos_hiv_faixa_55_65       |false      |
|bronze  |obitos_hiv_faixa_55_65_fem   |false      |
|bronze  |obitos_hiv_faixa_55_65_masc  |false 